In [1]:
import re

def limpiar_fallo(texto):
    # 1. Eliminar encabezados numerados o frases iniciales
    texto = re.sub(r'(?i)\b(Vistos:|Primero:|Segundo:|Tercero:|Cuarto:|Quinto:|Sexto:|Séptimo:|Octavo:|Noveno:|Décimo:|Y vistos, además,?)\s*', '', texto)

    # 2. Eliminar frases finales procesales
    texto = re.sub(r'Redacción de.*?\.', '', texto, flags=re.DOTALL)
    texto = re.sub(r'No firma el Fiscal Judicial.*?funcionario\.', '', texto, flags=re.DOTALL)
    texto = re.sub(r'Regístrese y devuélvase\.', '', texto)
    texto = re.sub(r'Rol Policía Local Nº.*?\.', '', texto)

    # 3. Normalizar saltos de línea: quitar líneas vacías dobles o múltiples
    texto = re.sub(r'\n{2,}', '\n', texto)

    # 4. Limpiar puntuación extraña (opcional)
    texto = re.sub(r'\s+([.,;:])', r'\1', texto)  # Eliminar espacio antes de puntuación
    texto = re.sub(r'\.\s+', '.\n', texto)        # Punto seguido de salto de línea para separar frases

    # 5. Eliminar espacios redundantes
    texto = re.sub(r'[ \t]+', ' ', texto)         # Espacios múltiples → 1
    texto = re.sub(r' *\n *', '\n', texto)        # Espacios al principio o fin de líneas

    return texto.strip()



def dividir_en_chunks(texto_limpio, max_longitud=1000):
    parrafos = [p.strip() for p in texto_limpio.split('\n') if len(p.strip()) > 0]
    chunks = []
    chunk_actual = ""

    for parrafo in parrafos:
        if len(chunk_actual) + len(parrafo) < max_longitud:
            chunk_actual += parrafo + " "
        else:
            chunks.append(chunk_actual.strip())
            chunk_actual = parrafo + " "

    if chunk_actual:
        chunks.append(chunk_actual.strip())

    return chunks

def extraer_articulos(texto):
    pattern = r'art[íi]culo[s]?\.?\s*(\d+[°º]?(\s*(bis|ter|quáter|quinquies)?)?)'
    encontrados = re.findall(pattern, texto, flags=re.IGNORECASE)
    # Solo quedarse con la parte del número
    articulos = [match[0].strip() for match in encontrados]
    return list(set(articulos)) if articulos else ["Ninguno"]


In [104]:
import os
import selenium.webdriver as webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv
from datetime import datetime

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'
edge_driver_path = os.path.join(os.getcwd(), 'msedgedriver.exe')
edge_service = Service(edge_driver_path)
edge_options = Options()
edge_options.add_argument(f'user-agent={user_agent}')
browser = webdriver.Edge(service= edge_service, options= edge_options)

browser.get("https://juris.pjud.cl/busqueda?Corte_de_Apelaciones")

time.sleep(10)

wait = WebDriverWait(browser, 10)
input_box = wait.until(EC.presence_of_element_located((By.ID, "tb_input_omnibox")))
input_box.click()
input_box.send_keys("19.496")
input_box.send_keys(Keys.ENTER)

time.sleep(5)
resultados_pagina = wait.until(EC.presence_of_element_located((By.ID, "resultados_busqueda_registros_por_pagina")))
select = Select(resultados_pagina)
select.select_by_value("50")




with open("Fallos_judiciales_ley_19.496(250).csv",mode="w", newline="", encoding="utf-8")as archivo:
    encabezados = ["Rol","Fecha_Sentencia","Corte de origen","Texto_sentencia","Leyes_mencionadas", "Artículos_mencionados"]

    writer = csv.DictWriter(archivo, fieldnames=encabezados)
    writer.writeheader()

    
    for i in range(5):
        avanzar_pag = wait.until(EC.presence_of_element_located((By.ID, "btnPaginador_pagina_adelante")))
        if i == 0:
            wait.until(EC.invisibility_of_element_located((By.ID, "capa_carga")))
            boton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[title="Ver"]')))
            boton.click()

        
        elif i>0:
            wait.until(EC.invisibility_of_element_located((By.ID, "capa_carga")))
            avanzar_pag.click()
            wait.until(EC.invisibility_of_element_located((By.ID, "capa_carga")))
            boton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[title="Ver"]')))
            boton.click()
        
        arreglar_boton_siguiente = (i>0)
        while True:
            time.sleep(5)
            
            div_sentencia = wait.until(EC.presence_of_element_located((By.ID,"contenedor_app_vue_detalle_sentencia")))
        
            wait.until(lambda driver: div_sentencia.text.strip() != "")

            if div_sentencia:
                capa_texto = div_sentencia.find_element(By.ID, "capa_texto_sentencia_202009111404")
                texto = capa_texto.get_attribute("innerText")
        
            if texto:
                Texto_sentencia = limpiar_fallo(texto)
                Leyes_mencionadas = re.findall(r'\bLey\s+N°\s+(\d+\.\d+)\b', Texto_sentencia)
                Leyes_unicas = list(set(Leyes_mencionadas))
                if not Leyes_unicas:
                     Leyes_unicas = "Ninguna" 
            
            else:
                print("Esta sentencia no tiene texto")
                Texto_sentencia = None
                Leyes_unicas = "Ninguna"
    
            datos_div = div_sentencia.find_element(By.CSS_SELECTOR, 'div[data-indice="datos_izquierdo"].card-body.h-50')
            if datos_div:
                
                datos_sentencia = datos_div.find_elements(By.CSS_SELECTOR, 'div[style="margin-bottom: 5px;"]')
                dato_rol = re.search(r'\b\d+-\d{4}\b',datos_sentencia[1].text.strip())
                if dato_rol:
                    resultado = dato_rol.group()
                    rol = resultado
                else:
                    print("Este fallo no tiene ROL")
                    Rol = ""
            
                dato_fecha = re.search(r'\b\d{2}-\d{2}-\d{4}\b',datos_sentencia[3].text.strip())
                if dato_fecha:
                    resultado1 = dato_fecha.group(0)
                    Fecha_Sentencia = datetime.strptime(resultado1, "%d-%m-%Y").date()
                else:
                    print("Este fallo no tiene fecha de sentencia")
                    Fecha_Sentencia = None

                dato_corte = re.search(r":(.+)",datos_sentencia[8].text.strip())
                if dato_corte:
                    resultado2 = dato_corte.group(1)
                    Corte_de_origen = resultado2
    
                else:
                    print("Este fallo no tiene Corte de origen")
                    Corte_de_origen = ""


                chunks = dividir_en_chunks(Texto_sentencia)
                articulos = extraer_articulos(Texto_sentencia)
                writer.writerow({
                    "Rol": rol,
                    "Fecha_Sentencia": Fecha_Sentencia,
                    "Corte de origen": Corte_de_origen,
                    "Texto_sentencia": chunks,
                    "Leyes_mencionadas": Leyes_unicas,
                    "Artículos_mencionados": "; ".join(articulos)
                 })

                div_opciones = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "col-md-4")))
                volver_busqueda = div_opciones.find_element(By.CSS_SELECTOR, '[title="Volver a la página de búsqueda"]')
                siguiente = div_opciones.find_element(By.ID, "detalle_sentencia_ir_siguiente")
                anterior = div_opciones.find_element(By.ID, "detalle_sentencia_ir_anterior")
                
                if siguiente.is_enabled():
                    wait.until(EC.invisibility_of_element_located((By.ID, "capa_carga")))
                    siguiente.click()
                    
                else:
                    wait.until(EC.invisibility_of_element_located((By.ID, "capa_carga")))
                    if arreglar_boton_siguiente:
                        anterior.click()
                        siguiente.click()
                        arreglar_boton_siguiente = False
                    else:
                        volver_busqueda.click()
                        break;
                        
                        
                    


browser.quit()

TimeoutException: Message: 


Miraremos un poco en el .csv que obtuvimos 

In [105]:
import pandas as pd
df = pd.read_csv("Fallos_judiciales_ley_19.496.csv")

df

,Rol,Fecha_Sentencia,Corte de origen,Texto_sentencia,Leyes_mencionadas,Artículos_mencionados
0,1734-2020,2023-01-06,C.A. de Santiago,"['Santiago, seis de enero de dos mil veintitré...","['18.287', '19.496']",26; 32
1,89455-2018,2019-03-28,C.A. de Santiago,"['', 'Santiago, veintiocho de marzo de dos mil...",Ninguna,19; 2°; 1545; 3°; 20
2,216-2006,2006-11-20,C.A. de Chillán,"['Chillán, veinte de noviembre de dos mil seis...",Ninguna,15; 50; 32; 24
3,4487-2019,2022-05-04,C.A. de Santiago,"['C.A. de Santiago Santiago, cuatro de mayo de...",Ninguna,50; 32; 58
4,70-2018,2019-05-29,C.A. de Iquique,['PODER JUDICIAL CORTE DE APELACIONES IQUIQUE ...,Ninguna,50; 145; 1; 1°; 58 bis; 3; 23; 12; 58; 24; 32
5,211-2024,2025-01-24,C.A. de Valdivia,"['Valdivia, veinticuatro de enero de dos mil v...",Ninguna,50; 2° Bis; 680; 1; 17; 2 bis; 543; 3
6,125-2012,2012-10-05,C.A. de Valdivia,"['Foja:134 Ciento Treinta y Cuatro VALDIVIA, c...",Ninguna,50; 144; 2 bis; 3; 32
7,42-2006,2006-11-02,C.A. de Rancagua,"['Rancagua, dos de noviembre de dos mil seis. ...",Ninguna,3º; 51; 1º; 44; 33; 2314; 2; 3; 12
8,288-2007,2008-03-28,C.A. de Punta Arenas,"['Punta Arenas, veintiocho de marzo de dos mil...",Ninguna,32; 23
9,938-2022,2024-04-17,C.A. de Santiago,"['C.A. de Santiago Santiago, diecisiete de abr...",Ninguna,1°; 50; 32; 58


In [106]:
import pandas as pd
df = pd.read_csv("Fallos_judiciales_ley_19.496(250).csv")

df

,Rol,Fecha_Sentencia,Corte de origen,Texto_sentencia,Leyes_mencionadas,Artículos_mencionados
0,1734-2020,2023-01-06,C.A. de Santiago,"['Santiago, seis de enero de dos mil veintitré...","['18.287', '19.496']",26; 32
1,89455-2018,2019-03-28,C.A. de Santiago,"['', 'Santiago, veintiocho de marzo de dos mil...",Ninguna,19; 2°; 1545; 3°; 20
2,216-2006,2006-11-20,C.A. de Chillán,"['Chillán, veinte de noviembre de dos mil seis...",Ninguna,15; 50; 32; 24
3,4487-2019,2022-05-04,C.A. de Santiago,"['C.A. de Santiago Santiago, cuatro de mayo de...",Ninguna,50; 32; 58
4,70-2018,2019-05-29,C.A. de Iquique,['PODER JUDICIAL CORTE DE APELACIONES IQUIQUE ...,Ninguna,50; 145; 1; 1°; 58 bis; 3; 23; 12; 58; 24; 32
...,...,...,...,...,...,...
95,243-2023,2024-03-11,C.A. de Valdivia,"['Valdivia, once de marzo de dos mil veinticua...","['18.287', '19.496']",144; 19; 14; 32
96,126-2023,2024-06-24,C.A. de Puerto Montt,"['Puerto Montt, veinticuatro de junio de dos m...","['19.880', '19.946', '19.496']",59 Bis; 3; 186; 58
97,20-2016,2016-10-13,C.A. de Coyhaique,"['Coyhaique, trece de Octubre de dos mil dieci...",Ninguna,6; 50; 2° bis; 2°; 1; 13; 2 bis; 1°; 2314; 23;...
98,191-2017,2017-09-22,C.A. de Valdivia,"['Valdivia, veintidós de septiembre de dos mil...","['20.416', '18.287', '19.496']",10; 17; 1; 7; 2 bis; 2; 12; 9; 14


In [90]:
_

9

In [91]:
i

2

In [98]:
aux

24